In [6]:

# Default locations if parameter not passed
benchmark_data_path = "/scratch/rodrigo.freitas/remy-schedulers/tb-compare.pkl"


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from rich import print as rprint
import seaborn as sns
import os
from scipy.stats import bootstrap

benchmark_data = pickle.load(open(benchmark_data_path, "rb"))

df = pd.DataFrame(benchmark_data["dataframe"])

sns.set_theme()

colors = [
    "#7f7f7f",  # grey
    "#621dac",  # main purple
    "#c5702d",  # orange
    "#000000",  # black,
    "#099892",  # teal
    "#ffd400",  # yellow
    "#7e57c4",  # pink/purple,
]
colors_backup = colors.copy()

def calculate_bootstrap_error(data):
    n_runs = benchmark_data["config"].metadata.runs
    if n_runs < 2:
        return 0
    res = bootstrap(
        (data,), np.mean, confidence_level=0.95, n_resamples=1000, method="basic"
    )
    return res.standard_error

In [12]:
bench_names = df["name"].unique()
print(f"Benchmarks found: {bench_names}")
run_metadata = benchmark_data["config"].applications
n_runs = benchmark_data["config"].metadata.runs

df_bench = df[df["name"] == 'tb-compare']


print(df_bench.columns)
print(df_bench.head())


Benchmarks found: ['tb-compare']
Index(['name', 'iter', 'full_output', 'tb_path', 'image', 'hosts',
       'ompc_tb_iter_num', 'tb-time', 'type', 'node_count', 'time'],
      dtype='object')
         name   iter                                        full_output  \
0  tb-compare   1000  [1749581463.931383] [sorgan-cpu1:236584:0]    ...   
1  tb-compare   1000  [1749581466.834971] [sorgan-cpu1:236845:0]    ...   
2  tb-compare  10000  [1749581467.543674] [sorgan-cpu1:236965:0]    ...   
3  tb-compare  10000  [1749581468.240650] [sorgan-cpu1:237088:0]    ...   
4  tb-compare   1000  [1749581468.938838] [sorgan-cpu1:237207:0]    ...   

        tb_path               image  \
0  tb-main-loop  scheduler-main.sif   
1  tb-main-loop  scheduler-main.sif   
2  tb-main-loop  scheduler-main.sif   
3  tb-main-loop  scheduler-main.sif   
4  tb-main-loop  scheduler-main.sif   

                                             hosts  ompc_tb_iter_num  \
0  sorgan-cpu1,sorgan-cpu2,sorgan-cpu3,sorgan-cpu4 

In [13]:
# print the field full_output of the first row
print("Full output of the first row:")
print(df_bench.iloc[0]["full_output"])

Full output of the first row:
[1749581463.931383] [sorgan-cpu1:236584:0]        ib_verbs.h:87   UCX  ERROR ibv_exp_query_device(mlx5_0) returned 95: Operation not supported
Running Task Benchmark
  Configuration:
    Task Graph 1:
      Time Steps: 32
      Max Width: 32
      Dependence Type: fft
      Radix: 3
      Period: 0
      Fraction Connected: 0.250000
      Kernel:
        Type: compute_bound
        Iterations: 1000
        Samples: 16
        Imbalance: 0.000000
      Output Bytes: 16
      Scratch Bytes: 0
Loop 0 : Elapsed time : 0.024403
Loop 1 : Elapsed time : 0.022082
Loop 2 : Elapsed time : 0.022360
Loop 3 : Elapsed time : 0.023046
Loop 4 : Elapsed time : 0.022887
Loop 5 : Elapsed time : 0.022918
Loop 6 : Elapsed time : 0.023012
Loop 7 : Elapsed time : 0.022698
Loop 8 : Elapsed time : 0.023097
Loop 9 : Elapsed time : 0.022702
Total Tasks 1024
Total Dependencies 2666
Total FLOPs 131137536
Total Bytes 0
Elapsed Time 2.360439e-01 seconds
FLOP/s 5.555641e+08
B/s 0.000000e